In [ ]:
%%capture
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import delfi.distribution as dd
import delfi.inference as infer
from delfi.inference import SNPEC as APT
import delfi.generator as dg

from delfi.utils.delfi2snl import SNLprior, SNLmodel

from delfi.simulator import TwoMoons
from delfi.summarystats import Identity
seed=46

# generator object (prior + simulator + summary statistics)
def init_g(seed):
    m = TwoMoons(mean_radius=0.1, sd_radius=0.01, baseoffset=0.25,
                 seed=seed)
    p = dd.Uniform(lower=[-1,-1], upper=[1,1], seed=seed)
    return dg.Default(model=m, prior=p, summary=Identity())

g = init_g(seed=seed)

# summary statistics of observed data
obs_stats = np.array([[0., 0.]])

# fitting setup
verbose =True
setup_dict = {}
setup_dict['seed'] = seed
setup_dict['obs_stats'] = obs_stats

# training schedule
n_rounds = 10
setup_dict['n_rounds'] = n_rounds
setup_dict['n_train'] = 1000

# fitting setup
setup_dict['minibatch'] = 100
setup_dict['reg_lambda'] = 0.001
setup_dict['pilot_samples'] = 0
setup_dict['prior_norm'] = False
setup_dict['init_norm'] = False

# grids for plotting posterior estimates
xo = 1.*obs_stats.flatten()
lims = np.array([[-1,1], [-1,1]])
i,j,resolution = 0,1,100
xx = np.linspace(lims[i, 0], lims[i, 1], resolution)
yy = np.linspace(lims[j, 0], lims[j, 1], resolution)
X, Y = np.meshgrid(xx, yy)
xy = np.concatenate(
    [X.reshape([-1, 1]), Y.reshape([-1, 1])], axis=1)


# SNPE A (always MDN)

In [ ]:
setup_dict_  = setup_dict.copy()
setup_dict_['n_components'] = 20
setup_dict_['n_hiddens'] = [50,50]
setup_dict_['svi'] = False
setup_dict_['val_frac'] = 0.1
setup_dict_['epochs'] = 500

# generator
g = init_g(seed=seed)

# inference object
res_A = infer.CDELFI(g, 
                 obs=obs_stats, 
                 n_hiddens=setup_dict_['n_hiddens'], 
                 n_components=setup_dict_['n_components'],
                 seed=seed, 
                 reg_lambda=setup_dict_['reg_lambda'],
                 pilot_samples=setup_dict_['pilot_samples'],
                 svi=setup_dict_['svi'],
                 verbose=verbose,
                 init_norm=setup_dict_['init_norm'],
                 prior_norm=setup_dict_['prior_norm'])

# train
logs_A, tds_A, posteriors_A = res_A.run(
                    n_train=setup_dict_['n_train'], 
                    n_rounds=setup_dict_['n_rounds'], 
                    val_frac=setup_dict_['val_frac'],
                    minibatch=setup_dict_['minibatch'], 
                    epochs=setup_dict_['epochs'])

In [ ]:
for r in range(len(posteriors_A)):
    posterior = posteriors_A[r]
    if posterior is None:
        pass
    else:
        plt.figure(figsize=(8, 8))
        pp = posterior.eval(xy, log=False).reshape(list(X.shape))
        plt.imshow(pp.T, origin='lower',
                   extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
                   aspect='auto', interpolation='none')
        plt.ylabel('SNPE-A')
        plt.xticks([])
        plt.yticks([])
        plt.title('N = ' + str( (r+1)*setup_dict_['n_train'] ))
        plt.show()        

# SNPE B (always MDN)

In [ ]:
setup_dict_ = setup_dict.copy()
setup_dict_['n_components'] = 20
setup_dict_['n_hiddens'] = [50,50]
setup_dict_['svi'] = True
setup_dict_['epochs'] = 500
setup_dict_['round_cl'] = 1

# generator
g = init_g(seed=seed)

# inference object
res_B = infer.SNPE(g, 
                 obs=obs_stats, 
                 n_hiddens=setup_dict_['n_hiddens'], 
                 n_components=setup_dict_['n_components'],
                 seed=seed, 
                 reg_lambda=setup_dict_['reg_lambda'],
                 pilot_samples=setup_dict_['pilot_samples'],
                 svi=setup_dict_['svi'],
                 verbose=verbose,
                 init_norm=setup_dict_['init_norm'],
                 prior_norm=setup_dict_['prior_norm'])

# train
logs_B, tds_B, posteriors_B = res_B.run(
                    n_train=setup_dict_['n_train'], 
                    n_rounds=setup_dict_['n_rounds'], 
                    minibatch=setup_dict_['minibatch'], 
                    round_cl=setup_dict_['round_cl'], 
                    epochs=setup_dict_['epochs'])

In [ ]:
for r in range(len(logs_B)):
    posterior = posteriors_B[r]
    if posterior is None:
        pass
    else:
        plt.figure(figsize=(8, 8))
        pp = posterior.eval(xy, log=False).reshape(list(X.shape))
        plt.imshow(pp.T, origin='lower',
                   extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
                   aspect='auto', interpolation='none')
        plt.ylabel('SNPE-B')
        plt.xticks([])
        plt.yticks([])
        plt.title('N = ' + str( (r+1)*setup_dict_['n_train'] ))
        plt.show()

# SNL (MAF)

In [ ]:
import sys
import snl.inference.nde as nde
from snl.ml.models.mafs import ConditionalMaskedAutoregressiveFlow

# MAF parameters
setup_dict_  = setup_dict.copy()
setup_dict_['mode'] = 'random'
setup_dict_['n_hiddens'] = [50,50]
setup_dict_['n_mades'] = 5
setup_dict_['act_fun'] = 'tanh'
setup_dict_['batch_norm'] = False # batch-normalization currently not supported
setup_dict_['train_on_all'] = True
setup_dict_['thin'] = 10
# control MAF seed
rng = np.random
rng.seed(seed)

# explicit call to MAF constructor
theta, x = g.gen(1)
n_inputs, n_outputs  = x.size, theta.size
model = ConditionalMaskedAutoregressiveFlow(
                n_inputs=n_inputs,
                n_outputs=n_outputs,
                n_hiddens=setup_dict_['n_hiddens'],
                act_fun=setup_dict_['act_fun'],
                n_mades=setup_dict_['n_mades'],
                mode=setup_dict_['mode'],
                rng=rng)

# generator
g = init_g(seed=seed)

# inference object
inf = nde.SequentialNeuralLikelihood(SNLprior(g.prior),               # method to draw parameters  
                                     SNLmodel(g.model, g.summary).gen # method to draw summary stats
                                    )

# train
rng = np.random # control  
rng.seed(seed)  # MCMC seed
model = inf.learn_likelihood(obs_stats.flatten(), model, n_samples=setup_dict_['n_train'], 
                             n_rounds=setup_dict_['n_rounds'], 
                             train_on_all=setup_dict_['train_on_all'], thin=setup_dict_['thin'], save_models=False, 
                             logger=sys.stdout, rng=rng)


In [ ]:
import snl.inference.mcmc as mcmc

# visualize learned likelihood
pp = model.eval((xo, xy), log=False).reshape(list(X.shape))
plt.imshow(pp.T, origin='lower',
           extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
           aspect='auto', interpolation='none')
plt.show()

# visualize learned posterior
log_posterior = lambda t: model.eval([t, obs_stats.flatten()]) + inf.prior.eval(t)
sampler = mcmc.SliceSampler(x=inf.all_ps[-3][-1], lp_f=log_posterior, thin=10)
ps = sampler.gen(1000)

plt.figure(figsize=(16,11))
for r in range(n_rounds-1):
    plt.subplot(np.ceil(n_rounds/3+1), 3, r + 1)
    plt.plot(inf.all_ps[r][:,0],
             inf.all_ps[r][:,1], 'k.')
    plt.axis([-1,1,-1,1])
    plt.xlabel('theta1')
    plt.xlabel('theta2')
    plt.title('round r='+str(r))

plt.subplot(np.ceil(n_rounds/3+1), 3, n_rounds+1)
plt.plot(ps[:,0],
         ps[:,1], 'k.')
plt.axis([-1,1,-1,1])
plt.xlabel('theta1')
plt.xlabel('theta2')
plt.title('round r='+str(n_rounds))
    
plt.show()

# APT (MDN)

In [ ]:

setup_dict_ = setup_dict.copy()
setup_dict_['proposal'] = 'gaussian'
setup_dict_['n_hiddens'] = [50,50]
setup_dict_['n_components'] = 20
setup_dict_['train_on_all'] = True
setup_dict_['svi'] = False
setup_dict_['n_null'] = setup_dict_['minibatch']-1
setup_dict_['epochs'] = 5000
setup_dict_['val_frac'] = 0.1


# generator
g = init_g(seed=seed)

# inference object
res_gC = APT(g, 
             obs=obs_stats, 
             n_hiddens=setup_dict_['n_hiddens'], 
             n_components=setup_dict_['n_components'],
             seed=seed, 
             reg_lambda=setup_dict_['reg_lambda'],
             pilot_samples=setup_dict_['pilot_samples'],
             svi=setup_dict_['svi'],
             verbose=verbose,
             prior_norm=setup_dict_['prior_norm'])

# train
logs_gC, tds_gC, posteriors_gC = res_gC.run(
                    n_train=setup_dict_['n_train'], 
                    proposal='gaussian',
                    n_rounds=setup_dict_['n_rounds'], 
                    minibatch=setup_dict_['minibatch'], 
                    epochs=setup_dict_['epochs'],
                    val_frac=setup_dict_['val_frac'],
                    train_on_all=setup_dict_['train_on_all'],    
                    verbose=True)

In [ ]:
for r in range(len(posteriors_gC)):

    posterior = posteriors_gC[r]
    if posterior is None:
        pass
    else:
        plt.figure(figsize=(8, 8))
    
        if not posterior is None:
            pp = posterior.eval(xy, log=False).reshape(list(X.shape))
            plt.imshow(pp.T, origin='lower',
                       extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
                       aspect='auto', interpolation='none')

        else:
            plt.text(-0.1, 0., 'broke', color='w')
            plt.imshow(np.zeros((resolution,resolution)), origin='lower',
                       extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
                       aspect='auto', interpolation='none')    

        plt.ylabel('continuous-proposal SNPE-C')
        plt.xticks([])
        plt.yticks([])
        plt.title('N = ' + str( (r+1)*setup_dict_['n_train'] ))
        plt.show()
        

# atomic-proposal APT (MAF)

In [ ]:
# MAF parameters
setup_dict_  = setup_dict.copy()
setup_dict_['proposal'] = 'discrete'
setup_dict_['moo'] = 'resample'
setup_dict_['mode'] = 'random'
setup_dict_['n_hiddens'] = [50,50]
setup_dict_['n_mades'] = 5
setup_dict_['act_fun'] = 'tanh'
setup_dict_['batch_norm'] = False # batch-normalization currently not supported
setup_dict_['train_on_all'] = True
setup_dict_['svi'] = False
setup_dict_['val_frac'] = 0.1
setup_dict_['n_null'] = setup_dict_['minibatch']-1

# control MAF seed
rng = np.random
rng.seed(seed)

# generator
g = init_g(seed=seed)

setup_dict_['epochs'] = 1000
if setup_dict_['train_on_all']:
    epochs=[setup_dict_['epochs'] // (r+1) for r in range(setup_dict_['n_rounds'])]
else:
    epochs=setup_dict_['epochs']

# inference object
res_dC = APT(g, 
             obs=obs_stats, 
             n_hiddens=setup_dict_['n_hiddens'],
             seed=seed,
             reg_lambda=setup_dict_['reg_lambda'],
             pilot_samples=setup_dict_['pilot_samples'],
             svi=setup_dict_['svi'],
             n_mades=setup_dict_['n_mades'],
             act_fun=setup_dict_['act_fun'],
             mode=setup_dict_['mode'],
             rng=rng,
             batch_norm=setup_dict_['batch_norm'],
             verbose=verbose,
             prior_norm=setup_dict_['prior_norm'])

# train
logs_dC, tds_dC, posteriors_dC = res_dC.run(
                    n_train=setup_dict_['n_train'],
                    proposal=setup_dict_['proposal'],
                    moo=setup_dict_['moo'],
                    n_null = setup_dict_['n_null'],
                    n_rounds=setup_dict_['n_rounds'],
                    train_on_all=setup_dict_['train_on_all'],
                    minibatch=setup_dict_['minibatch'],
                    val_frac=setup_dict_['val_frac'],
                    epochs=epochs)



In [ ]:
plt.figure(figsize=(9, 12))
plt.subplot(np.ceil(n_rounds/3), 3, 1)

for r in range(len(posteriors_dC)):
    plt.subplot(np.ceil(n_rounds/3), 3, r + 1)
    posterior = posteriors_dC[r] 
    pp = posterior.eval(xy, log=False).reshape(list(X.shape))
    plt.imshow(pp.T, origin='lower',
               extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
               aspect='auto', interpolation='none')
    plt.title('posterior estimate after round r='+str(r+1))

plt.show()

## SMC

In [ ]:
from util import rejSMC # SMC that ensures proposed parameters are sampled only within prior support

g = init_g(seed=seed)

sampler = rejSMC(SNLprior(dd.Uniform(lower=[-1,-1], upper=[1,1])), SNLmodel(g.model, g.summary).gen)

# run with default eps_decay from Papamapakarios et al. (2018)
p_smc, log_weights = sampler.run(obs_data=obs_stats.flatten(), 
                              n_particles=1000, eps_init=1, eps_last=0.1, eps_decay=0.9)

plt.plot(p_smc[:,0], p_smc[:,1], '.')
plt.show()

# assemble figure

In [ ]:
from snl.pdfs import gaussian_kde # using kernel density estimates to plot density estimates for SNL & SMC

plt.figure(figsize=(9, 12), frameon=False)
idx_r = [0,4,9] # plot results after rounds 1, 5 and 10


###########
# SNPE-A  #
###########

for r in range(len(idx_r)):
    plt.subplot(5, 4, 1 +r)
    try:
        posterior = posteriors_A[idx_r[r]] 
        if not posterior is None:
            pp = posterior.eval(xy, log=False).reshape(list(X.shape))
            plt.imshow(pp.T, origin='lower',
                       extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
                       aspect='auto', interpolation='none')
    except:
        plt.imshow(np.zeros((resolution,resolution)), origin='lower',
                   extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
                   aspect='auto', interpolation='none')
        plt.text(-0.1, 0., 'broke', color='w')
    if r == 0:
        plt.ylabel('SNPE-A')
    plt.xticks([])
    plt.yticks([])
    plt.title('N = ' + str( (idx_r[r]+1)*setup_dict['n_train']))


###########
# SNPE-B  #
###########


for r in range(len(idx_r)):
    plt.subplot(5, 4, 5 +r)
    try:
        posterior = posteriors_B[idx_r[r]] 
        pp = posterior.eval(xy, log=False).reshape(list(X.shape))
        plt.imshow(pp.T, origin='lower',
                   extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
                   aspect='auto', interpolation='none')
    except:
        pass
    if r == 0:
        plt.ylabel('SNPE-B')
    plt.xticks([])
    plt.yticks([])


############
# MDN APT  #
############   
    
for r in range(len(idx_r)):
    plt.subplot(5, 4, 9 +r)
    try:
        posterior = posteriors_gC[idx_r[r]] 
        pp = posterior.eval(xy, log=False).reshape(list(X.shape))
        plt.imshow(pp.T, origin='lower',
                   extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
                   aspect='auto', interpolation='none')
    except:
        pass
    if r == 0:
        plt.ylabel('APT (MDN)')
    plt.xticks([])
    plt.yticks([])
    
    
############
# MAF APT  #
############  

for r in range(len(idx_r)):
    plt.subplot(5, 4, 13 +r)
    try:
        posterior = posteriors_dC[idx_r[r]] 
        pp = posterior.eval(xy, log=False).reshape(list(X.shape))
        plt.imshow(pp.T, origin='lower',
                   extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
                   aspect='auto', interpolation='none')
    except:
        pass        
    if r == 0:
        plt.ylabel('APT (MAF)')
    plt.xticks([])
    plt.yticks([])
    

############
#   SNL    #
############      
    
for r in range(len(idx_r)):
    plt.subplot(5, 4, 17 +r)
    try:
        if idx_r[r]+1 < len(inf.all_ps) :
            plt.plot(inf.all_ps[idx_r[r]+1][:,0],
                     inf.all_ps[idx_r[r]+1][:,1], 'k.')
        else:     
            plt.plot(ps[:,0],
                     ps[:,1], 'k.')
    except:
        pass
    plt.axis([lims[0][0], lims[0][1], lims[1][0], lims[1][1]])
        
    if r == 0:
        plt.ylabel('SNL')
    plt.xticks([])
    plt.yticks([])
    

############
#   SMC    #
############      
        
try:    
    plt.subplot(5,4,12)
    i,j = 0,1
    kde = gaussian_kde(xs=p_smc, std=0.01)
    kde = dd.MoG(xs = [dd.Gaussian(m = kde.xs[i].m, S=kde.xs[i].S) for i in range(p_smc.shape[0])], 
                 a=1./p_smc.shape[0] * np.ones(p_smc.shape[0]))
    pp = kde.eval(xy, log=False).reshape(list(X.shape))
    plt.imshow(pp.T, origin='lower',
               extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
               aspect='auto', interpolation='none')
    plt.axis([lims[0][0], lims[0][1], lims[1][0], lims[1][1]])    
    plt.xticks([])
    plt.yticks([])
    plt.ylabel('SMC')
except:
    pass

plt.show()